### Матричные факторизации

В данной работе вам предстоит познакомиться с практической стороной матричных разложений.
Работа поделена на 4 задания:
1. Вам необходимо реализовать SVD разложения используя SGD на explicit данных
2. Вам необходимо реализовать матричное разложения используя ALS на implicit данных
3. Вам необходимо реализовать матричное разложения используя BPR(pair-wise loss) на implicit данных
4. Вам необходимо реализовать матричное разложения используя WARP(list-wise loss) на implicit данных


In [1]:
import implicit
import pandas as pd
import numpy as np
import scipy.sparse as sp
from tqdm import trange

from lightfm.datasets import fetch_movielens

c:\users\265573\appdata\local\programs\python\python38\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


В данной работе мы будем работать с explicit датасетом movieLens, в котором представленны пары user_id movie_id и rating выставленный пользователем фильму

Скачать датасет можно по ссылке https://grouplens.org/datasets/movielens/1m/

In [2]:
ratings = pd.read_csv('data/ml-1m/ratings.dat', delimiter='::', header=None, 
        names=['user_id', 'movie_id', 'rating', 'timestamp'], 
        usecols=['user_id', 'movie_id', 'rating'], engine='python')

In [3]:
movie_info = pd.read_csv('data/ml-1m/movies.dat', delimiter='::', header=None, 
        names=['movie_id', 'name', 'category'], engine='python')

In [4]:
ratings['user_id'] -= 1
ratings['movie_id'] -= 1
movie_info['movie_id'] -= 1

Explicit данные

In [5]:
ratings.head(10)

,user_id,movie_id,rating
0,0,1192,5
1,0,660,3
2,0,913,3
3,0,3407,4
4,0,2354,5
5,0,1196,3
6,0,1286,5
7,0,2803,5
8,0,593,4
9,0,918,4


Для того, чтобы преобразовать текущий датасет в Implicit, давайте считать что позитивная оценка это оценка >=4

In [6]:
implicit_ratings = ratings.loc[(ratings['rating'] >= 4)]

In [7]:
implicit_ratings.head(10)

,user_id,movie_id,rating
0,0,1192,5
3,0,3407,4
4,0,2354,5
6,0,1286,5
7,0,2803,5
8,0,593,4
9,0,918,4
10,0,594,5
11,0,937,4
12,0,2397,4


Удобнее работать с sparse матричками, давайте преобразуем DataFrame в CSR матрицы

In [8]:
users = implicit_ratings["user_id"]
movies = implicit_ratings["movie_id"]
user_item = sp.coo_matrix((np.ones_like(users), (users, movies)))
user_item_t_csr = user_item.T.tocsr()
user_item_csr = user_item.tocsr()

В качестве примера воспользуемся ALS разложением из библиотеки implicit

Зададим размерность латентного пространства равным 64, это же определяет размер user/item эмбедингов

In [9]:
model = implicit.als.AlternatingLeastSquares(factors=64, iterations=100, calculate_training_loss=True)

В качестве loss здесь всеми любимый RMSE

In [10]:
model.fit(user_item_t_csr)

  0%|          | 0/100 [00:00<?, ?it/s]

Построим похожие фильмы по 1 movie_id = Истории игрушек

In [11]:
movie_info.head(5)

,movie_id,name,category
0,0,Toy Story (1995),Animation|Children's|Comedy
1,1,Jumanji (1995),Adventure|Children's|Fantasy
2,2,Grumpier Old Men (1995),Comedy|Romance
3,3,Waiting to Exhale (1995),Comedy|Drama
4,4,Father of the Bride Part II (1995),Comedy


In [12]:
get_similars = lambda item_id, model : pd.concat([movie_info[movie_info["movie_id"] == x[0]] 
                                        for x in model.similar_items(item_id)], axis=0)

Как мы видим, симилары действительно оказались симиларами.

Качество симиларов часто является хорошим способом проверить качество алгоритмов.

P.S. Если хочется поглубже разобраться в том как разные алгоритмы формируют разные латентные пространства, рекомендую загружать полученные вектора в tensorBoard и смотреть на сформированное пространство

In [13]:
get_similars(0, model)

,movie_id,name,category
0,0,Toy Story (1995),Animation|Children's|Comedy
3045,3113,Toy Story 2 (1999),Animation|Children's|Comedy
2286,2354,"Bug's Life, A (1998)",Animation|Children's|Comedy
33,33,Babe (1995),Children's|Comedy|Drama
584,587,Aladdin (1992),Animation|Children's|Comedy|Musical
2315,2383,Babe: Pig in the City (1998),Children's|Comedy
360,363,"Lion King, The (1994)",Animation|Children's|Musical
1526,1565,Hercules (1997),Adventure|Animation|Children's|Comedy|Musical
2252,2320,Pleasantville (1998),Comedy
2692,2760,"Iron Giant, The (1999)",Animation|Children's


Давайте теперь построим рекомендации для юзеров

Как мы видим юзеру нравится фантастика, значит и в рекомендациях ожидаем увидеть фантастику

In [14]:
get_user_history = lambda user_id, dataset : pd.concat([movie_info[movie_info["movie_id"] == x] for x in dataset[dataset["user_id"] == user_id]["movie_id"]], axis=0)

In [15]:
get_user_history(3, implicit_ratings)

,movie_id,name,category
3399,3467,"Hustler, The (1961)",Drama
2882,2950,"Fistful of Dollars, A (1964)",Action|Western
1196,1213,Alien (1979),Action|Horror|Sci-Fi|Thriller
1023,1035,Die Hard (1988),Action|Thriller
257,259,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi
1959,2027,Saving Private Ryan (1998),Action|Drama|War
476,479,Jurassic Park (1993),Action|Adventure|Sci-Fi
1180,1197,Raiders of the Lost Ark (1981),Action|Adventure
1885,1953,Rocky (1976),Action|Drama
1081,1096,E.T. the Extra-Terrestrial (1982),Children's|Drama|Fantasy|Sci-Fi


Получилось! 

Мы действительно порекомендовали пользователю фантастику и боевики, более того встречаются продолжения тех фильмов, которые он высоко оценил

In [16]:
get_recommendations = lambda user_id, model, dataset : pd.concat([movie_info[movie_info["movie_id"] == x[0]]
                                               for x in model.recommend(user_id, dataset)], axis=0)

In [17]:
get_recommendations(3, model, user_item_csr)

,movie_id,name,category
585,588,Terminator 2: Judgment Day (1991),Action|Sci-Fi|Thriller
1284,1303,Butch Cassidy and the Sundance Kid (1969),Action|Comedy|Western
2502,2570,"Matrix, The (1999)",Action|Sci-Fi|Thriller
1271,1290,Indiana Jones and the Last Crusade (1989),Action|Adventure
1178,1195,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Drama|Sci-Fi|War
1182,1199,Aliens (1986),Action|Sci-Fi|Thriller|War
1884,1952,"French Connection, The (1971)",Action|Crime|Drama|Thriller
1892,1960,Rain Man (1988),Drama
957,968,"African Queen, The (1951)",Action|Adventure|Romance|War
3458,3526,Predator (1987),Action|Sci-Fi|Thriller


Теперь ваша очередь реализовать самые популярные алгоритмы матричных разложений

Что будет оцениваться:
1. Корректность алгоритма
2. Качество получившихся симиларов
3. Качество итоговых рекомендаций для юзера

Base MF class

In [18]:
class MF:
    def __init__(self, size, factors, reg_param, max_iter, user_label='user_id', item_label='movie_id', target_label='rating'):
        self.users_sz = size[0]
        self.items_sz = size[1]
        self.factors = factors
        
        self.max_iter = max_iter
        self.reg_param = reg_param
        
        self.users_m = np.random.uniform(0.0, 1 / np.sqrt(factors), (self.users_sz, factors))
        self.items_m = np.random.uniform(0.0, 1 / np.sqrt(factors), (self.items_sz, factors))
        
        self.user_label = user_label
        self.item_label = item_label
        self.target_label = target_label
        
    def similar_items(self, item_id, amount=10):
        distances = np.linalg.norm(self.items_m - self.items_m[item_id], axis=1)
        return list(zip(np.argsort(distances), list(sorted(distances))))[:amount]
    
    def recommend(self, user_id, data, amount=10):
        ratings = self.users_m[user_id] @ self.items_m.T
        non_zeros = data[user_id].nonzero()[1]
        return list(filter(lambda x: x[0] not in non_zeros, zip(np.argsort(ratings), list(sorted(ratings)))))[-amount:]



### Задание 1. Не использую готовые решения, реализовать SVD разложение используя SGD на explicit данных

In [21]:
class SVD(MF):
    def __init__(self, size, factors, learning_rate, reg_param, max_iter, user_bias_param=1e-6, item_bias_param=1e-6, user_label='user_id', item_label='movie_id', target_label='rating'):
        super().__init__(size, factors, reg_param, max_iter, user_label, item_label, target_label)
        
        self.learning_rate = learning_rate
        self.user_bias_param = user_bias_param
        self.item_bias_param = item_bias_param
        
    def fit(self, train):
        self.users_bias = np.zeros(self.users_sz)
        self.items_bias = np.zeros(self.items_sz)
        self.mu = train[self.target_label].mean()
        
        with trange(self.max_iter, desc='Learning...', leave=True) as t:
            for i in t:
            
                user, item, rating = train.iloc[np.random.randint(len(train))][:]
                
                error = (self.users_m[user, :] @ self.items_m[item, :] + self.users_bias[user] + self.items_bias[item] + self.mu) - rating
                
                self.users_bias[user] -= self.learning_rate * (error + self.user_bias_param * self.users_bias[user])
                self.items_bias[item] -= self.learning_rate * (error + self.item_bias_param * self.items_bias[item])
                
                self.users_m[user, :] -= self.learning_rate * (error * self.items_m[item, :] + self.reg_param * self.users_m[user, :])
                self.items_m[item, :] -= self.learning_rate * (error * self.users_m[user, :] + self.reg_param * self.items_m[item, :])
            
                if i % 10000 == 0:
                    self.A = (self.users_m @ self.items_m.T) + self.users_bias[:,np.newaxis] + self.items_bias + self.mu
                    measured = self.A[train[self.user_label], train[self.item_label]]
                    rmse = np.sqrt(np.mean((measured - train[self.target_label].to_numpy())**2))
                    t.set_description("RMSE: " + str(rmse))
                    t.refresh()
                
        self.A = (self.users_m @ self.items_m.T) + self.users_bias[:, np.newaxis] + self.items_bias + self.mu
            
            
        

In [51]:
svd = SVD((np.max(ratings['user_id']) + 1, np.max(ratings['movie_id']) + 1), factors=64, learning_rate=1e-2, reg_param=1e-5, max_iter=int(1e7))

svd.fit(ratings)

RMSE: 0.6839319080067887: 100%|██████████████████████████████████████████| 10000000/10000000 [46:33<00:00, 3580.12it/s]


In [52]:
get_similars(0, svd)

,movie_id,name,category
0,0,Toy Story (1995),Animation|Children's|Comedy
3045,3113,Toy Story 2 (1999),Animation|Children's|Comedy
3090,3158,Fantasia 2000 (1999),Animation|Children's|Musical
2020,2088,"Rescuers Down Under, The (1990)",Animation|Children's
2225,2293,Antz (1998),Animation|Children's
2021,2089,"Rescuers, The (1977)",Animation|Children's
1000,1012,"Parent Trap, The (1961)",Children's|Drama
3348,3416,"Crimson Pirate, The (1952)",Adventure|Comedy|Sci-Fi
942,953,Mr. Smith Goes to Washington (1939),Drama
1838,1906,Mulan (1998),Animation|Children's


In [53]:
users = ratings["user_id"]
movies = ratings["movie_id"]
explicit_user_item_csr = sp.coo_matrix((np.ones_like(users), (users, movies))).tocsr()

In [54]:
get_user_history(3, ratings)

,movie_id,name,category
3399,3467,"Hustler, The (1961)",Drama
1192,1209,Star Wars: Episode VI - Return of the Jedi (1983),Action|Adventure|Romance|Sci-Fi|War
2882,2950,"Fistful of Dollars, A (1964)",Action|Western
1196,1213,Alien (1979),Action|Horror|Sci-Fi|Thriller
1023,1035,Die Hard (1988),Action|Thriller
257,259,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi
1959,2027,Saving Private Ryan (1998),Action|Drama|War
476,479,Jurassic Park (1993),Action|Adventure|Sci-Fi
1178,1195,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Drama|Sci-Fi|War
1180,1197,Raiders of the Lost Ark (1981),Action|Adventure


In [55]:
get_recommendations(3, svd, explicit_user_item_csr)

,movie_id,name,category
1888,1956,Chariots of Fire (1981),Drama
908,919,Gone with the Wind (1939),Drama|Romance|War
2641,2709,"Blair Witch Project, The (1999)",Horror
911,922,Citizen Kane (1941),Drama
907,918,"Wizard of Oz, The (1939)",Adventure|Children's|Drama|Musical
3026,3094,"Grapes of Wrath, The (1940)",Drama
1166,1182,"English Patient, The (1996)",Drama|Romance|War
3091,3159,Magnolia (1999),Drama
1262,1281,Fantasia (1940),Animation|Children's|Musical
900,911,Casablanca (1942),Drama|Romance|War


### Задание 2. Не использую готовые решения, реализовать матричное разложение используя ALS на implicit данных

In [27]:

class ALS(MF):
    def fit(self, train):
        
        train_arr = train.toarray()
        rows, cols = train.nonzero()
        
        reg_matrix = self.reg_param * sp.identity(self.factors)
        
        identity_items = sp.identity(self.items_sz)
        identity_users = sp.identity(self.users_sz)
        
        
        with trange(self.max_iter, desc='Learning...', leave=True) as t:
            self.A = self.users_m @ self.items_m.T
            measured = self.A[rows, cols]
            rmse = np.sqrt(np.mean((measured - train_arr[rows, cols])**2))
            t.set_description("RMSE: " + str(rmse))
            t.refresh()
                
            for i in t:
                
                items_t = self.items_m.T
                item_x_itemt = items_t @ self.items_m
                for user in range(self.users_sz):
                    cur_user_ratings = train_arr[user, :]
                    
                    confidence_m = sp.diags(10 * cur_user_ratings)
                    Cu = items_t @ sp.csr_matrix.dot(confidence_m, self.items_m)
                    self.users_m[user, :] = np.linalg.inv(item_x_itemt + Cu + reg_matrix) @ items_t @ (confidence_m + identity_items) @ cur_user_ratings
                
                users_t = self.users_m.T
                user_x_usert = users_t @ self.users_m
                
                for item in range(self.items_sz):
                    cur_item_users = train_arr[:, item]
                    
                    confidence_m = sp.diags(10 * cur_item_users)
                    Ci = users_t @ sp.csr_matrix.dot(confidence_m, self.users_m)
                    self.items_m[item, :] = np.linalg.inv(user_x_usert + Ci + reg_matrix) @ users_t @ (confidence_m + identity_users) @ cur_item_users
                    
                
                self.A = self.users_m @ self.items_m.T
                measured = self.A[rows, cols]
                rmse = np.sqrt(np.mean((measured - train_arr[rows, cols])**2))
                t.set_description("RMSE: " + str(rmse))
                t.refresh()

In [28]:
als = ALS(user_item_csr.shape, factors=64, reg_param=1e-2, max_iter=5)

als.fit(user_item_csr)

RMSE: 0.30370529174990063: 100%|████████████████████████████████████████████████████████| 5/5 [14:09<00:00, 169.86s/it]


In [29]:
get_similars(0, als)

,movie_id,name,category
0,0,Toy Story (1995),Animation|Children's|Comedy
3045,3113,Toy Story 2 (1999),Animation|Children's|Comedy
584,587,Aladdin (1992),Animation|Children's|Comedy|Musical
1245,1264,Groundhog Day (1993),Comedy|Romance
2286,2354,"Bug's Life, A (1998)",Animation|Children's|Comedy
2327,2395,Shakespeare in Love (1998),Comedy|Romance
33,33,Babe (1995),Children's|Comedy|Drama
2252,2320,Pleasantville (1998),Comedy
360,363,"Lion King, The (1994)",Animation|Children's|Musical
352,355,Forrest Gump (1994),Comedy|Romance|War


In [30]:
get_user_history(3, implicit_ratings)

,movie_id,name,category
3399,3467,"Hustler, The (1961)",Drama
2882,2950,"Fistful of Dollars, A (1964)",Action|Western
1196,1213,Alien (1979),Action|Horror|Sci-Fi|Thriller
1023,1035,Die Hard (1988),Action|Thriller
257,259,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi
1959,2027,Saving Private Ryan (1998),Action|Drama|War
476,479,Jurassic Park (1993),Action|Adventure|Sci-Fi
1180,1197,Raiders of the Lost Ark (1981),Action|Adventure
1885,1953,Rocky (1976),Action|Drama
1081,1096,E.T. the Extra-Terrestrial (1982),Children's|Drama|Fantasy|Sci-Fi


In [31]:
get_recommendations(3, als, user_item_csr)

,movie_id,name,category
1284,1303,Butch Cassidy and the Sundance Kid (1969),Action|Comedy|Western
3634,3702,Mad Max 2 (a.k.a. The Road Warrior) (1981),Action|Sci-Fi
108,109,Braveheart (1995),Action|Drama|War
1271,1290,Indiana Jones and the Last Crusade (1989),Action|Adventure
1192,1209,Star Wars: Episode VI - Return of the Jedi (1983),Action|Adventure|Romance|Sci-Fi|War
453,456,"Fugitive, The (1993)",Action|Thriller
1182,1199,Aliens (1986),Action|Sci-Fi|Thriller|War
2502,2570,"Matrix, The (1999)",Action|Sci-Fi|Thriller
1178,1195,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Drama|Sci-Fi|War
585,588,Terminator 2: Judgment Day (1991),Action|Sci-Fi|Thriller


### Задание 3. Не использую готовые решения, реализовать матричное разложение BPR на implicit данных

In [32]:
from collections import defaultdict
from numpy.random import choice, randint

class BPR(MF):
    def __init__(self, size, factors, learning_rate, reg_param, max_iter, user_label='user_id', item_label='movie_id', target_label='rating'):
        super().__init__(size, factors, reg_param, max_iter, user_label, item_label, target_label)
        
        self.learning_rate = learning_rate
        
    def fit(self, train):
        train_arr = train.toarray()
        
        positives = defaultdict(list)
        rows, cols = train.nonzero()
        for i, j in zip(rows, cols):
            positives[i].append(j)
        
        
        with trange(self.max_iter, desc='Learning...', leave=True) as t:
            for i in t:
                for user in range(self.users_sz):
                
                    for i in positives[user]:
                        j = self.get_j(user, positives)
                
                        rating = (self.users_m[user, :] @ self.items_m[i, :].T) - (self.users_m[user, :] @ self.items_m[j, :].T)
                        
                        exponent = np.exp(-rating)
                        dsigmoid = exponent / (1 + exponent)
                
                        self.users_m[user, :] =  self.users_m[user, :] + self.learning_rate * (dsigmoid * (self.items_m[i, :] - self.items_m[j, :]) + self.reg_param * self.users_m[user, :])
                        self.items_m[i, :] =  self.items_m[i, :] + self.learning_rate * (dsigmoid * self.users_m[user, :] + self.reg_param * self.items_m[i, :])
                        self.items_m[j, :] =  self.items_m[j, :] + self.learning_rate * (-dsigmoid * self.users_m[user, :] + self.reg_param * self.items_m[j, :])
                
                self.A = self.users_m @ self.items_m.T
                measured = self.A[rows, cols]
                rmse = np.sqrt(np.mean((measured - train_arr[rows, cols])**2))
                t.set_description("RMSE: " + str(rmse))
                t.refresh()
                
            self.A = self.users_m @ self.items_m.T
        
    
    def get_j(self, user, positives):
        j = randint(self.items_sz)
        while j in positives[user]:
            j = randint(self.items_sz)
        return j
                
                
                
        
        
        
        

In [33]:
bpr = BPR(user_item_csr.shape, factors=64, learning_rate=1e-2, reg_param=1e-5, max_iter=150)

bpr.fit(user_item_csr)

RMSE: 6.638018921623734: 100%|█████████████████████████████████████████████████████| 150/150 [3:28:54<00:00, 83.56s/it]


In [34]:
get_similars(0, bpr)

,movie_id,name,category
0,0,Toy Story (1995),Animation|Children's|Comedy
3045,3113,Toy Story 2 (1999),Animation|Children's|Comedy
2286,2354,"Bug's Life, A (1998)",Animation|Children's|Comedy
2252,2320,Pleasantville (1998),Comedy
33,33,Babe (1995),Children's|Comedy|Drama
1245,1264,Groundhog Day (1993),Comedy|Romance
360,363,"Lion King, The (1994)",Animation|Children's|Musical
2225,2293,Antz (1998),Animation|Children's
584,587,Aladdin (1992),Animation|Children's|Comedy|Musical
1468,1499,Grosse Pointe Blank (1997),Comedy|Crime


In [35]:
get_recommendations(3, bpr, user_item_csr)

,movie_id,name,category
315,317,"Shawshank Redemption, The (1994)",Drama
1182,1199,Aliens (1986),Action|Sci-Fi|Thriller|War
1192,1209,Star Wars: Episode VI - Return of the Jedi (1983),Action|Adventure|Romance|Sci-Fi|War
2502,2570,"Matrix, The (1999)",Action|Sci-Fi|Thriller
108,109,Braveheart (1995),Action|Drama|War
589,592,"Silence of the Lambs, The (1991)",Drama|Thriller
585,588,Terminator 2: Judgment Day (1991),Action|Sci-Fi|Thriller
1178,1195,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Drama|Sci-Fi|War
1203,1220,"Godfather: Part II, The (1974)",Action|Crime|Drama
847,857,"Godfather, The (1972)",Action|Crime|Drama


### Задание 4. Не использую готовые решения, реализовать матричное разложение WARP на implicit данных

In [47]:
class WARP(BPR):
    def __init__(self, size, factors, learning_rate, reg_param, max_iter, max_sampled, user_label='user_id', item_label='movie_id', target_label='rating'):
        super().__init__(size, factors, learning_rate, reg_param, max_iter, user_label, item_label, target_label)
        
        self.max_sampled = max_sampled

    def get_j(self, user, positives):
        j = randint(self.items_sz)
        highest_rating = self.users_m[user, :] @ self.items_m[j, :].T
        for _ in range(self.max_sampled):
            candidate = randint(self.items_sz)
            while candidate in positives[user]:
                candidate = randint(self.items_sz)
            candidate_rating = self.users_m[user, :] @ self.items_m[candidate, :].T
            if highest_rating < candidate_rating:
                highest_rating = candidate_rating
                j = candidate
        return j

In [48]:
warp = WARP(user_item_csr.shape, factors=64, learning_rate=1e-3, reg_param=1e-3, max_iter=40, max_sampled=50)

warp.fit(user_item_csr)

RMSE: 0.7395524154660724: 100%|███████████████████████████████████████████████████| 40/40 [19:30:33<00:00, 1755.84s/it]


In [49]:
get_similars(0, warp)

,movie_id,name,category
0,0,Toy Story (1995),Animation|Children's|Comedy
3045,3113,Toy Story 2 (1999),Animation|Children's|Comedy
360,363,"Lion King, The (1994)",Animation|Children's|Musical
2692,2760,"Iron Giant, The (1999)",Animation|Children's
2012,2080,"Little Mermaid, The (1989)",Animation|Children's|Comedy|Musical|Romance
3184,3252,Wayne's World (1992),Comedy
2286,2354,"Bug's Life, A (1998)",Animation|Children's|Comedy
1058,1072,Willy Wonka and the Chocolate Factory (1971),Adventure|Children's|Comedy|Fantasy
584,587,Aladdin (1992),Animation|Children's|Comedy|Musical
591,594,Beauty and the Beast (1991),Animation|Children's|Musical


In [50]:
get_recommendations(3, warp, user_item_csr)

,movie_id,name,category
2650,2718,"Haunting, The (1999)",Horror|Thriller
315,317,"Shawshank Redemption, The (1994)",Drama
1649,1695,Bent (1997),Drama|War
604,607,Fargo (1996),Crime|Drama|Thriller
71,71,Kicking and Screaming (1995),Comedy|Drama
1192,1209,Star Wars: Episode VI - Return of the Jedi (1983),Action|Adventure|Romance|Sci-Fi|War
2502,2570,"Matrix, The (1999)",Action|Sci-Fi|Thriller
589,592,"Silence of the Lambs, The (1991)",Drama|Thriller
1178,1195,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Drama|Sci-Fi|War
2789,2857,American Beauty (1999),Comedy|Drama
